### Importing Modules

## First Part

In [ ]:
# Part-1 Data Processing and Cleaning
# Importing modules
import csv
import sqlite3
from sqlite3 import Error



# Connecting to our database
connection = sqlite3.connect('risk_index.db')
# Creating a cursor object to execute
# SQL queries on a database table
cursor = connection.cursor()

# ****** Creating Risk Table **************
# Table Definition
create_table_risk = '''CREATE TABLE risk(
				country TEXT NOT NULL,
				wri REAL NOT NULL,
				vul REAL NOT NULL,
				year INTEGER NOT NULL)
				'''
# Creating the table into our
# database
cursor.execute(create_table_risk)
# Opening the csv file
risk_file = open('data/risk.csv')
# Reading the contents of the
# csv file
risk_contents = csv.reader(risk_file)
# SQL query to insert data into the
# risk table
insert_records_risk = "INSERT INTO risk (country, wri, vul, year) VALUES(?, ?, ?, ?)"
# Importing the contents of the file
# into our person table
cursor.executemany(insert_records_risk, risk_contents)
# Committing the changes
connection.commit()


# ****** Creating temperature Table **************
# Table Definition
create_table_temp = '''CREATE TABLE temperature(
				country TEXT NOT NULL,
				temp REAL NOT NULL,
				year INTEGER NOT NULL)
				'''
# Creating the table into our
# database
cursor.execute(create_table_temp)
# Opening the csv file
temp_file = open('data/temp.csv')
# Reading the contents of the
# csv file
temp_contents = csv.reader(temp_file)
# SQL query to insert data into the
# risk table
insert_records_temp = "INSERT INTO temperature (country, year, temp) VALUES(?, ?, ?)"
# Importing the contents of the file
# into our person table
cursor.executemany(insert_records_temp, temp_contents)
# Committing the changes
connection.commit()

# ****** Creating 3rd Final table wrisk **************
# The purpose of creating wrisk table is to 
# combine data from both risk and temperature into one table
# using LEFT JOIN operator

# Table Definition
create_table_wrisk = '''CREATE TABLE wrisk(
				country TEXT NOT NULL,
				wri REAL NOT NULL,
				vul REAL NOT NULL,
				year INTEGER NOT NULL,
				temp REAL NOT NULL
)
				'''
# Creating the table into our
# database
cursor.execute(create_table_wrisk)

# creating sql query for left join of risk and temperature table
join_table = '''
INSERT INTO wrisk
SELECT risk.country, wri, vul, risk.year, temp
FROM risk
LEFT JOIN temperature
ON risk.country=temperature.country AND risk.year=temperature.year;
'''
cursor.execute(join_table)
connection.commit()
# closing the database connection
connection.close()

print("**********************  Process Done - Table Created [risk, temperature, and wrisk]  **********************")


# Part 2: Application Structure
TO run part 2, open the "testcode.py" file in python IDE, and run each function individually.

Specifically, the application performs 5 operations which are shown below.

## First Function: Display all data
![display](data/img/img1.png)

## 2nd Function: Insert data into database
![insert](data/img/img2.png)

## 3rd Function: Update data from database
![update](data/img/img3.png)

## 4th Function: Delete data from database
![delete](data/img/img4.png)

## 5th Function: Search data fronm database
![search](data/img/img5.png)

In [42]:
# This code block (Part-2) is here just for review, 
# to run this open testcode.py file, and run each line individually while other commented

# importing modules
import tkinter as tk
import sqlite3
from tkinter import ttk, messagebox

# connecting to our earlier created database
con = sqlite3.connect("risk_index.db")

# additional command in case no previous input is supplied, the code will still work
con.execute("CREATE TABLE IF NOT EXISTS wrisk(country TEXT, wri REAL, vul REAL, year INTEGER, temp REAL);")

# designing insert data query
def insert_data(country, wri, vul, year, temp):
    conn = sqlite3.connect("risk_index.db")
    conn.execute("INSERT INTO risk(country, wri, vul, year, temp) VALUES( '" + country + "', '" + wri +"', '" + vul + "', '" + year + "', '" + temp + "' );")
    conn.commit()
    conn.close()
    messagebox.showinfo("Success", "Data Saved Successfully.")

# designing insert window of application
def insert():
    add_window = tk.Tk()
    add_window.title("Add Details")
    tk.Label(add_window).grid(row=0, column=0, columnspan=2)
    tk.Label(add_window, text="country:").grid(row=1, column=0)
    country_entry = tk.Entry(add_window, width=50)
    country_entry.grid(row=1, column=1, padx=25)
    tk.Label(add_window, text="wri:").grid(row=2, column=0)
    wri_entry = tk.Entry(add_window, width=50)
    wri_entry.grid(row=2, column=1, padx=25)
    tk.Label(add_window, text="vul:").grid(row=3, column=0)
    vul_entry = tk.Entry(add_window, width=50)
    vul_entry.grid(row=3, column=1, padx=25)
    tk.Label(add_window, text="year:").grid(row=4, column=0, padx=20)
    year_entry = tk.Entry(add_window, width=50)
    year_entry.grid(row=4, column=1, padx=25)
    tk.Label(add_window, text="temp:").grid(row=5, column=0)
    grade_entry = tk.Entry(add_window, width=50)
    grade_entry.grid(row=5, column=1, padx=25)

    tk.Button(add_window, text='Submit', activebackground='grey', activeforeground='white', command=lambda: submit()).grid(row=6, column=0, columnspan=2, pady=10)
    # define working of submit button
    def submit():
        country = country_entry.get()
        wri = wri_entry.get()
        vul = vul_entry.get()
        year = str(year_entry.get())
        temp = str(grade_entry.get())
        insert_data(country, wri, vul, year, temp)
        add_window.destroy()

    add_window.mainloop()

    #designing display window for application 
def display():
    connn = sqlite3.connect("risk_index.db")
    display_window = tk.Tk()
    display_window.title("World Risk Index Database")
    table = ttk.Treeview(display_window)
    table["columns"] = ("one", "two", "three", "four", "five")

    table.heading("one", text="Country")
    table.heading("two", text="WRI")
    table.heading("three", text="Vulnerability")
    table.heading("four", text="Year")
    table.heading("five", text="Temperature Change")

    cursor = connn.execute("SELECT rowid,* FROM wrisk")
    i = 0
    for row in cursor:
        table.insert('', i, text="Risk " + str(row[0]), values=(row[1], row[2], row[3], row[4], row[5]))
        i = i + 1
    table.pack()
    connn.close()

# designing update window for application
def update():
    update_window = tk.Tk()
    update_window.title("Update Details")
    tk.Label(update_window, text="Enter the country to be Updated:").grid(row=0, column=0, sticky="W", padx=10, columnspan=2)
    s_id = tk.Entry(update_window, width=50)
    s_id.grid(row=1, column=0, sticky="W", padx=10, columnspan=2)
    tk.Label(update_window, text="\nEnter the new values:").grid(row=2, column=0, sticky="W", padx=10, pady=10, columnspan=2)
    tk.Label(update_window, text="country:").grid(row=3, column=0, sticky="W", padx=10, pady=10)
    s_country = tk.Entry(update_window, width=50)
    s_country.grid(row=3, column=1, sticky="W", padx=10, pady=10)
    tk.Label(update_window, text="wri:").grid(row=4, column=0, sticky="W", padx=10, pady=10)
    s_wri = tk.Entry(update_window, width=50)
    s_wri.grid(row=4, column=1, sticky="W", padx=10, pady=10)
    tk.Label(update_window, text="vul:").grid(row=5, column=0, sticky="W", padx=10, pady=10)
    s_vul = tk.Entry(update_window, width=50)
    s_vul.grid(row=5, column=1, sticky="W", padx=10, pady=10)
    tk.Label(update_window, text="year No:").grid(row=6, column=0, sticky="W", padx=10, pady=10)
    s_year = tk.Entry(update_window, width=50)
    s_year.grid(row=6, column=1, sticky="W", padx=10, pady=10)
    tk.Label(update_window, text="temp").grid(row=7, column=0, sticky="W", padx=10, pady=10)
    s_temp = tk.Entry(update_window, width=50)
    s_temp.grid(row=7, column=1, sticky="W", padx=10, pady=10)
    tk.Button(update_window, text="Update", activebackground='grey', activeforeground='white',
              command=lambda: submit()).grid(row=8, column=0, padx=10, pady=10, columnspan=2)
    # submit for update
    def submit():
        sid = s_id.get()
        scountry = s_country.get()
        swri = s_wri.get()
        svul = s_vul.get()
        syear = s_year.get()
        stemp = s_temp.get()
        scon = sqlite3.connect("risk_index.db")
        scon.execute("UPDATE wrisk SET country = '" + scountry + "',wri = '" + swri + "', vul = '" + svul +
                     "', year = '" + syear + "', temp = '" + stemp + "' WHERE rowid = " + sid + ";")
        scon.commit()
        scon.close()
        messagebox.showinfo("Success", "Data Updated Successfully.")
        update_window.destroy()
    update_window.mainloop()

# designing delete window for application
def delete():
    delete_window = tk.Tk()
    delete_window.title("Delete Info ")
    tk.Label(delete_window, text="Enter country whose details are to be removed:").grid(row=0, column=0, padx=10, pady=10)
    d_country = tk.Entry(delete_window, width=50)
    d_country.grid(row=0, column=1, padx=10, pady=10)
    tk.Button(delete_window, text="Delete Details", activebackground='grey', activeforeground='white',
              command=lambda: submit()).grid(row=1, column=0, columnspan=2)
    tk.Label(delete_window).grid(row=2, column=0, columnspan=2)
    # submit button for delete
    def submit():
        dcountry = d_country.get()
        dcon = sqlite3.connect("risk_index.db")
        dcon.execute("DELETE FROM wrisk WHERE country = '" + dcountry+"';")
        dcon.commit()
        dcon.execute("VACUUM;")
        dcon.commit()
        dcon.close()
        messagebox.showinfo("Success", "Deleted Successfully.")
        delete_window.destroy()
    delete_window.mainloop()

# designing search window for application
def search():
    search_window = tk.Tk()
    search_window.title("Search Risk Index Details")

    tk.Label(search_window, text="Enter the country whose details are to be searched:").grid(row=0, column=0,
                                                                                                     padx=10, pady=10)
    f_country = tk.Entry(search_window, width=50)
    f_country.grid(row=0, column=1, padx=10, pady=10)

    tk.Label(search_window, text="Results:").grid(row=1, column=0, sticky="W", columnspan=2, padx=10, pady=10)

    tk.Button(search_window, text="Search", activebackground='grey', activeforeground='white',
              command=lambda: submit()).grid(row=2, column=0, columnspan=2)
    tk.Label(search_window).grid(row=3, column=0, sticky="W", columnspan=2, padx=10, pady=10)
    details = ttk.Treeview(search_window)
    details["columns"] = ("one", "two", "three", "four", "five")

    details.heading("one", text="country")
    details.heading("two", text="wri")
    details.heading("three", text="vul")
    details.heading("four", text="year No")
    details.heading("five", text="temp")
    # submit button for search
    def submit():
        for row in details.get_children():
            details.delete(row)

        fcountry = f_country.get()
        fcon = sqlite3.connect("risk_index.db")
        cursor = fcon.execute("SELECT rowid,* from wrisk, WHERE country = '" + fcountry + "';")
        fcon.commit()

        i = 0
        for row in cursor:
            details.insert('', i, text="wrisk " + str(row[0]), values=(row[1], row[2], row[3], row[4], row[5]))
            i = i + 1

        details.grid(row=4, column=0, columnspan=2, padx=10, pady=10)
        fcon.close()
    search_window.mainloop()


con.close()


# Part 3 World Risk Index Template

## Please provide first the year and country name and then run the code. 

In [41]:
# inputs
i_year = 2019
i_count = 'Pakistan'


#- -----------------------------------------
# filter all by country
import sqlite3
from sqlite3 import Error

# Connecting to our database
connection = sqlite3.connect('risk_index.db')
# Creating a cursor object to execute
# SQL queries on a database table
cursor = connection.cursor()

# ****** Creating Risk Table **************
# Table Definition
template_filter = '''
SELECT *
From wrisk
WHERE country like? and year =?

				'''
# Creating the table into our
# database
cursor.execute(template_filter,(i_count, i_year,))
# Opening the csv file

rows = cursor.fetchall()

from pandas import DataFrame
df = DataFrame(rows)

country = df[0]
tempC = df[4]
year = df[3]
wri = df[1]
vul = df[2]

template = 'The temperature change of {} is {} for the year {}. Furthermore, the WorldRiskIndex (WRI) is {} and Vulnerability is {}'.format(country.values[0], tempC.values[0], year.values[0], wri.values[0], vul.values[0])
print(template)

The temperature change of Pakistan is 1.021 for the year 2019. Furthermore, the WorldRiskIndex (WRI) is 7.08 and Vulnerability is 56.52
